In [12]:
# !pip install newsapi-python

In [13]:
from newsapi import NewsApiClient
import json
# import secrets from secrets.json file
with open('secrets.json') as f:
    secrets = json.load(f) # secrets is a dictionary

api_key = secrets['API_KEY_NEWS'] # News API key

# Init
newsapi = NewsApiClient(api_key=api_key)
import datetime

# Get the start date of the last month
now = datetime.datetime.now()
last_month_start = now - datetime.timedelta(days=now.day)

# Set the 'from_param' parameter to the start date of the last month
all_articles = newsapi.get_everything(q='Funny',
                                      from_param=last_month_start.strftime("%Y-%m-%d"),
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)


Creating Monikers Cards from the articles...

In [14]:
articles = all_articles['articles']

cards = []
for article in articles:
    title = article['title']
    description = article['description']
    card = {
        'title': title,
        'description': description,
        'point_value': 1 # add logic to determine point value based on either obscurity of the phrase or the number of other articles that use the phrase in the title or description.
    }
    cards.append(card)

# show one card
cards[2]


{'title': 'Elon Musk is a Jekyll and Hyde character. And as head of Twitter, Hyde is winning | John Naughton',
 'description': 'On the one hand, he’s a creative genius. On the other, he’s destroying a key debating chamberWatching what’s going on at Twitter is like watching a guy losing his mind in slow motion. The guy in question is Elon Musk, who once upon a time was the world’s rich…',
 'point_value': 1}

In [16]:
# !pip install sumy

  Using cached sumy-0.11.0-py2.py3-none-any.whl (97 kB)
  Using cached breadability-0.1.20-py2.py3-none-any.whl
  Using cached pycountry-22.3.5-py2.py3-none-any.whl
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ed38854d12c2ca03b10552d84f7418f8711e521d31f5ec917f567f44c4ec0e2b
  Stored in directory: /Users/grahamwaters/Library/Caches/pip/wheels/4e/29/5e/e5f6f558e24e0d70f6808e3e8fdaf2c04ff597233098afc550
Successfully built docopt


In [17]:
import os
import openai
import json
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def summarize_text(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return summary

text = "This is some text that I want to summarize. It can be as long or as short as needed. The summary will be a few sentences long."
num_sentences = 2
summary = summarize_text(text, num_sentences)

for sentence in summary:
    print(sentence)


This is some text that I want to summarize.
The summary will be a few sentences long.


In [40]:
import re

num_words = 5
for card in cards:
    title = card['title']
    description = card['description']

    # Check if title or description is None before calling re.sub
    if title is not None:
        title = re.sub('<[^<]+?>', '', title)
        card['title'] = title
    if description is not None:
        description = re.sub('<[^<]+?>', '', description)
        card['description'] = description

    text = f"{title} {description}"
    summary = summarize_text(text, num_words)
    summary_text = " ".join([str(sentence) for sentence in summary])
    card['summary'] = summary_text
    print("------------------")
    print(title)
    print("------------------")
    print(summary_text)


------------------
The Patient review – Steve Carell is absolutely faultless in this intense kidnap thriller
------------------
The Patient review – Steve Carell is absolutely faultless in this intense kidnap thriller The Office star plays a kidnapped therapist in this claustrophobic two-hander with Domhnall Gleeson. It might be slow going, but the performances are utterly stellarYears ago, any foray by a comedian into drama was greeted with anything from horror to mockery…
------------------
Robbie Coltrane remembered by Miriam Margolyes
------------------
Robbie Coltrane remembered by Miriam Margolyes 30 March 1950 – 14 October 2022His friend and fellow actor pays tribute to a stupendous talent with a rollicking, expansive, occasionally cruel spirit – and a vulnerable side he never let people seeDavid Trimble remembered by Jonathan PowellR…
------------------
Elon Musk is a Jekyll and Hyde character. And as head of Twitter, Hyde is winning | John Naughton
------------------
Elon Musk

In [30]:
cards[5]

{'title': 'I was well looked after at the Qatar World Cup – and that makes me feel deeply uncomfortable | Emma Kemp',
 'description': 'Each time a worker opened a door for me with a subservient smile, I felt a grotesque pang of privilegeSometimes, during or after a big sporting event, journalists are asked to write a diary-esque piece about their experiences covering said event in whichever …',
 'point_value': 1,
 'summary': 'I was well looked after at the Qatar World Cup – and that makes me feel deeply uncomfortable | Emma Kemp Each time a worker opened a door for me with a subservient smile, I felt a grotesque pang of privilegeSometimes, during or after a big sporting event, journalists are asked to write a diary-esque piece about their experiences covering said event in whichever …'}

In [41]:
!pip install spacy
!pip install wikipedia
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 10.3 MB/s eta 0:00:0000:0100:01
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached cymem-2.0.7-cp39-cp39-macosx_10_9_x86_64.whl (32 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.6/776.6 kB 25.3 MB/s eta 0:00:00
  Using cached murmurhash-1.0.9-cp39-cp39-macosx_10_9_x86_64.whl (18 kB)
  Using cached spacy_legacy-3.0.10-py2.py3-none-any.whl (21 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached srsly-2.4.5-cp39-cp39-macosx_10_9_x86_64.whl (491 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached preshed-3.0.8-cp39-cp39-macosx_10_9_x86_64.whl (107 kB)
  Using cached confection-0.0.3-py3-none-any.whl (32 kB)
  Using cached blis-0.7.9-cp39-cp39-macosx_10_9_x86_64.whl (6.1 MB)


In [ ]:
import spacy
from spacy.tokens import Token, Span
